In [8]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api.exceptions import TMDbException
import random
import pandas as pd
import csv
import numpy as np
from math import exp
import ast
tmdb=TMDb()
tmdb.api_key='9cf68f4c97c8f0cc6bb9646da389a808'

In [9]:
# Initialize csv file paths
csv_file_path= 'merged_data_mod_scores_filtered_US_release_year_star_scores.csv'
# Create DataFrames from both CSV files
df = pd.read_csv(csv_file_path)


In [ ]:
df['Movie Contribution to Director and Production Scores'] = 0

#Calculates the contribution of a movie to a star score based on whether or not it was a top 100 domestic grossing movie of that year

release_years = range(2010, 2024)
#iterates through the years
for year in release_years:
    df_year = df[df["Release Year"] == year]
    print(df_year.head())
    #sorts by revenue (descending)
    df_year = df_year.sort_values(by=['Merged Revenue'], ascending=False)
    #iterate through top 100 movies, give them points based on the ranking (100 to the top grossing, 99 to the 2nd top, ..., 1 to the 100th)
    for i in range(0,100):
        tmdb_id_to_update = df_year['IMDB ID'].iloc[i]
        #record the contribution in the dataframe
        df.loc[df['IMDB ID'] == tmdb_id_to_update, 'Movie Contribution to Director and Production Scores'] = 100 - i

In [11]:
# Create a DataFrame with unique production company IDs
prod_ids = []

for index, row in df.iterrows():
    prod_ids_str = row['Production Company ID']
    
    # Check for NaN values and skip them
    if pd.isna(prod_ids_str):
        continue

    # Safely evaluate the content of 'prod_ids' if it's not NaN
    prod_ids += ast.literal_eval(prod_ids_str)

unique_prod_ids = list(set(prod_ids))

# Create a list of years from 2011 to 2023
years = [str(year) for year in range(2010, 2024)]

# Initialize the data with zeros
data = {f'star_{year}': [0] * len(unique_prod_ids) for year in years}
data['ids'] = unique_prod_ids

# Create the 'output_df' DataFrame
prod_df = pd.DataFrame(data)

# Reorder columns with 'ids' as the first column
prod_df = prod_df[['ids'] + [col for col in prod_df.columns if col != 'ids']]

# Print the first few rows of the 'output_df' DataFrame for debugging
print(prod_df.head())

   ids  star_2010  star_2011  star_2012  star_2013  star_2014  star_2015  \
0    1          0          0          0          0          0          0   
1    2          0          0          0          0          0          0   
2    3          0          0          0          0          0          0   
3    4          0          0          0          0          0          0   
4    5          0          0          0          0          0          0   

   star_2016  star_2017  star_2018  star_2019  star_2020  star_2021  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   star_2022  star_2023  
0          0          0  
1          0          0  
2          0          0  
3          0

In [12]:
# Iterate through df to calculate production company scores
for index, row in df.iterrows():
    prod_ids_str = row['Production Company ID']
    # Check for NaN values and skip them
    if pd.isna(prod_ids_str):
        continue
    # Safely evaluate the content of 'prod_ids' if it's not NaN
    prod_ids = ast.literal_eval(prod_ids_str)
    release_year = row['Release Year']
    score_contribution = row['Movie Contribution to Director and Production Scores']
    # Iterate through each prod_id in the prod_ids array
    for prod_id in prod_ids:
        # Find the corresponding row in prod_df
        prod_df_row = prod_df[prod_df['ids'] == prod_id]
        if not prod_df_row.empty:
            prod_score = prod_df_row[f'star_{release_year}'].values[0]
            # Add the contribution from the movie to the total star score for that company for that year
            if not pd.isna(score_contribution):
                prod_score += score_contribution
            # Assign the updated production company score to the corresponding 'star_yyyy' column
            prod_df.loc[prod_df['ids'] == prod_id, f'star_{release_year}'] = prod_score


output_prods = prod_df.copy(deep = True)           

#sums the star scores for the previous three years (beginning in 2012)
for prod_id in prod_df['ids']:
    for column in prod_df.columns:
        if column.startswith("star_"):
            release_year = int(column.split("_")[1])
            if release_year > 2011:
                previous_year = release_year - 1
                year_before_previous = release_year - 2
                prod_df_row = prod_df[prod_df['ids'] == prod_id]
                if not prod_df_row.empty:
                    prod_score = prod_df_row[column].values[0]
                    # Calculate score from the previous year
                    previous_year_score = prod_df_row[f'star_{previous_year}'].values[0]
                    prod_score += previous_year_score
                    # Calculate score from the year before the previous year
                    year_before_previous_score = prod_df_row[f'star_{year_before_previous}'].values[0]
                    prod_score += year_before_previous_score
                    # Assign the updated director score to the corresponding 'star_year' column
                    output_prods.loc[output_prods['ids'] == prod_id, column] = prod_score


In [13]:
output_prods.to_csv('prod_company_df.csv', index=False)

In [14]:
# Create an output dataframe
df_output = df.copy()  # Copy the original DataFrame
df_output = df_output[df_output['Release Year'] > 2012]
df_output['Total Production Company Score'] = 0  # create an empty star scores column
df_output['Avg Production Company Score'] = 0  # create an empty star scores column

for index, row in df_output.iterrows():
    row_prod_info = row['Production Company ID']  # Extract prod_ids array
    if not pd.isna(row_prod_info):  # Check for NaN
        row_prod_info = ast.literal_eval(row_prod_info)
    else:
        row_prod_info = []  # Set to an empty list if NaN
    #computes sum of production company scores
    for prod_id in row_prod_info:
        release_date = row['Release Year']
        previous_year = release_date - 1
        prod_info_row = output_prods[output_prods['ids'] == prod_id]  # Get the row containing the queried prod id
        star_column_name = f'star_{previous_year}'  # get the star score column for the previous year
        star_score = prod_info_row[star_column_name].values[0]  # Use the previous year as the star score
        if star_score != None:
            df_output.loc[df['IMDB ID'] == row['IMDB ID'], 'Total Production Company Score'] += star_score
    #computes avg of production company scores
    if len(row_prod_info) > 0:
        df_output.loc[df['IMDB ID'] == row['IMDB ID'], 'Avg Production Company Score'] = df_output.loc[df['IMDB ID'] == row['IMDB ID'], 'Total Production Company Score'] / len(row_prod_info)


# Save to csv
print(df_output.head())
df_output.to_csv('merged_data_mod_scores_filtered_US_release_year_star_scores_prod_scores.csv', index=False)

   Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0 Running Time  \
0             0             0           0  1 hr 22 min   
1             1             1           2  1 hr 49 min   
2             2             2           3  1 hr 30 min   
3             3             3           4  1 hr 46 min   
4             4             4           6  1 hr 29 min   

                                           Genres               IMDB Title  \
0                               Documentary Sport     Into the Mind (2013)   
1             Action Crime Drama Mystery Thriller       Broken City (2013)   
2                                           Drama   Redwood Highway (2013)   
3                     Biography Drama History War  Bless Me, Ultima (2012)   
4  Action Adventure Comedy Horror Sci-Fi Thriller           Spiders (2013)   

    MPAA      Domestic Distributor  Domestic Opening  \
0    NaN                       NaN               NaN   
1      R     Twentieth Century Fox         8268908.0   
2  PG-13      